In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

! ls
! pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive	sample_data
/content


In [2]:
! cd '/content/gdrive/My Drive/Colab_Temp'
! ls

gdrive	sample_data


In [0]:
# Get resources dataset from Jiatai's Git
#! git clone https://github.com/emilwallner/Coloring-greyscale-images
! git clone https://github.com/hjt486/datasets-colornet

# Check the folder for resources and get path
#! ls
#! pwd

fatal: destination path 'datasets-colornet' already exists and is not an empty directory.


In [0]:
# Force to install Keras 2.1.5 due to save\load model issue
!pip install keras==2.1.5
# Keras
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import TensorBoard
# Image pre/post processing
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
# Misc
import math
import numpy as np
import os
import random
import tensorflow as tf
from IPython.display import Image, display
from google.colab import files

# Create snapshot saver
saver = tf.train.Saver()
saver.save(session, CHECKPOINT_PATH)

# Define resource root path
path = '/content/datasets-colornet/testdata/'

Using TensorFlow backend.


In [0]:
# Get images
X = []
file_limit = os.listdir(path + 'Train/Mixed/')[:100]
for filename in file_limit:
    X.append(img_to_array(load_img(path + 'Train/Mixed/' + filename)))
X = np.array(X, dtype=float)



# Set up train and test data
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Training parameters
total = 100
batch_size = 100
epochs = 5000
steps = math.ceil(total/batch_size)

# Generate training data
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

FileNotFoundError: ignored

In [0]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse', metrics=['acc'])

In [0]:
# Train model      
tensorboard = TensorBoard(log_dir="output/first_run")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=epochs, steps_per_epoch=steps)

In [0]:
# Get test images
image_path = path + 'Validate/1dZTRG.jpg'
image = img_to_array(load_img(image_path))
color_me = np.array(image, dtype=float)
color_me = rgb2lab(1.0/255*image)[:,:,0]
color_me = color_me.reshape(1, 256, 256, 1)

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
cur = np.zeros((256, 256, 3))
cur[:,:,0] = color_me[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))
Image('img_result.png')

In [0]:
# Save Model
model.save('mixed.h5')
print('Model saved 100x1x5000')
files.download('mixed.h5') 
! ls